In [1]:
import cv2
from imutils import face_utils
import pandas as pd
import numpy as np
import mediapipe as mp

import os
import re
import math


In [64]:
pack_identify = {
    "mouth_up_all" : [61, 78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308, 291],
    "mouth_up_indicate" : [61, 78, 308, 291],
    "mouth_near_ext" : [78, 308],
    "mouth_left_ext" : 61,
    "mouth_rigth_ext" : 291,

    "left_brow" : [70, 63, 105, 66, 107, 55, 65],
    "rigth_brow" : [336, 296, 334, 293, 300, 285, 295],
    "left_brow_indicate" : [70, 107],
    "left_brow_center" : 105,
    "rigth_brow_indicate" : [336, 300],
    "rigth_brow_center" : 334,


    "jawline_all" : [234, 93, 132, 58, 172, 136, 150, 149, 176, 148, 152, 377, 400, 378, 379, 365, 397],
    "jawline_indicate" : [149, 176, 148, 152, 377, 400, 378],
    "jawline_center" : [148, 152, 377],
    "jawline_indicate" : [149, 378]
}

In [57]:
class Landmark():
    def __init__(self, x : int, y : int):
        self.x = x
        self.y = y

In [61]:
def segment_length(x1, y1, x2, y2):
    return math.sqrt((y2 - y1) ** 2 + (x2 - x1) ** 2)


def point_to_line_distance(x0, y0, x1, y1, x2, y2):
    numerator = abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
    denominator = segment_length(x1, y1, x2, y2)
    if denominator == 0:
        raise ValueError("Точки x1,y1 и x2,y2 совпадают — нельзя построить прямую.")
    return numerator / denominator, denominator / 2


In [ ]:
def dev_mouth(landmarks : list[Landmark],
              left_ext_idx : int = 61,
              rigth_ext_idx : int = 291,
              near_lms : list = pack_identify["mouth_near_ext"]):
    left_ext_lm = landmarks[left_ext_idx]
    rigth_ext_lm = landmarks[rigth_ext_idx]

    left_near_lm = landmarks[near_lms[0]]
    rigth_near_lm = landmarks[near_lms[1]]
    
    left_div_y = abs(left_ext_lm.y - left_near_lm.y)
    rigth_div_y = abs(rigth_ext_lm.y - rigth_near_lm.y)

    left_div_x = abs(left_ext_lm.x - left_near_lm.x) if left_ext_lm.x != left_near_lm.x else 1
    rigth_div_x = abs(rigth_ext_lm.x - rigth_near_lm.x) if rigth_ext_lm.x != rigth_near_lm.x else 1

    result_div = (left_div_y / left_div_x + rigth_div_y / rigth_div_x) / 2 * 100
    return result_div

def dev_brows(landmarks : list[Landmark],
                left_ext_idxs : list = pack_identify["left_brow_indicate"],
                rigth_ext_idxs : list = pack_identify["rigth_brow_indicate"],
                left_center_idx : int = pack_identify["left_brow_center"],
                rigth_center_idx : int = pack_identify["rigth_brow_center"]):
    left_dist, left_half_base_len = point_to_line_distance(landmarks[left_center_idx].x,
                                       landmarks[left_center_idx].y,
                                       landmarks[left_ext_idxs[0]].x,
                                       landmarks[left_ext_idxs[0]].y,
                                       landmarks[left_ext_idxs[1]].x,
                                       landmarks[left_ext_idxs[1]].y)
    
    rigth_dist, rigth_half_base_len = point_to_line_distance(landmarks[rigth_center_idx].x,
                                       landmarks[rigth_center_idx].y,
                                       landmarks[rigth_ext_idxs[0]].x,
                                       landmarks[rigth_ext_idxs[0]].y,
                                       landmarks[rigth_ext_idxs[1]].x,
                                       landmarks[rigth_ext_idxs[1]].y)

    result_div = (left_dist / left_half_base_len + rigth_dist / rigth_half_base_len) / 2 * 100
    
    return result_div

def dev_jawline(landmarks : list[Landmark],
                center : list[int] = pack_identify["jawline_center"],
                ext_idx : list[int] = pack_identify["jawline_indicate"]):
    left_dist, _ = point_to_line_distance(landmarks[center[0]].x,
                                       landmarks[center[0]].y,
                                       landmarks[ext_idx[0]].x,
                                       landmarks[ext_idx[0]].y,
                                       landmarks[ext_idx[1]].x,
                                       landmarks[ext_idx[1]].y)
    
    rigth_dist, _ = point_to_line_distance(landmarks[center[-1]].x,
                                       landmarks[center[-1]].x,
                                       landmarks[ext_idx[0]].x,
                                       landmarks[ext_idx[0]].y,
                                       landmarks[ext_idx[1]].x,
                                       landmarks[ext_idx[1]].y)
    
    left_dist_x = abs(landmarks[center[0]].x - landmarks[ext_idx[0]].x)
    rigth_dist_x = abs(landmarks[center[-1]].x - landmarks[ext_idx[-1]].x)

    result_div = (left_dist / left_dist_x + rigth_dist / rigth_dist_x) / 2 * 100

    return result_div
   


In [66]:
DATASET_ALL_DATA = "../all_data/"

# Загружаем модель
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)



In [67]:
def numeric_key(name):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', name)]

In [75]:
def get_landmarks(image_path):
    image_array = np.fromfile(image_path, dtype=np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    if image is None:
        raise ValueError("Изображение не загружено!")
            
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    height, width = image_rgb.shape[:2]

    # Детектируем лицо
    faces = face_mesh.process(image_rgb)

    if not faces.multi_face_landmarks:
        return

    shape = faces.multi_face_landmarks[0]

    return [Landmark(p.x * width, p.y * height) for p in shape.landmark]

In [76]:
def build_dataframe(dataset_dir,
                    t_procents_mouth: list,
                    f_procents_mouth: list,
                    t_procents_brows: list,
                    f_procents_brows: list,
                    t_procents_jawline: list,
                    f_procents_jawline: list,
                    get_label_func = lambda k : k // 8): # получаем числовой код папки
    k = 0
    for label_dir in sorted(os.listdir(dataset_dir), key=numeric_key):
        label_path = os.path.join(dataset_dir, label_dir)
        if not os.path.isdir(label_path):
            continue

        for filename in os.listdir(label_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(label_path, filename)

                result = get_landmarks(image_path)
                
                if result is None:
                    continue
                landmarks = result
                
                avg_dev_percent_mouth = dev_mouth(
                    landmarks = landmarks)
                
                avg_dev_percent_brows = dev_brows(
                    landmarks = landmarks)
                
                avg_dev_percent_jawline = dev_jawline(
                    landmarks = landmarks)
                
                if get_label_func(k) % 2 == 0:
                    t_procents_mouth.append(avg_dev_percent_mouth)
                    t_procents_brows.append(avg_dev_percent_brows)
                    t_procents_jawline.append(avg_dev_percent_jawline)

                else:
                    f_procents_mouth.append(avg_dev_percent_mouth)
                    f_procents_brows.append(avg_dev_percent_brows)
                    f_procents_jawline.append(avg_dev_percent_jawline)
                
        k += 1


In [77]:
t_procents_mouth = []
f_procents_mouth = []
t_procents_brows = []
f_procents_brows = []
t_procents_jawline = []
f_procents_jawline = []


In [78]:
build_dataframe(DATASET_ALL_DATA,
                t_procents_mouth=t_procents_mouth,
                f_procents_mouth=f_procents_mouth,
                t_procents_brows=t_procents_brows,
                f_procents_brows=f_procents_brows,
                t_procents_jawline=t_procents_jawline,
                f_procents_jawline=f_procents_jawline,)


KeyboardInterrupt: 

In [ ]:
print(len(t_procents_mouth))
print(len(f_procents_mouth))

print(len(t_procents_jawline))
print(len(f_procents_jawline))

print(len(t_procents_brows))
print(len(f_procents_brows))


37705
31426
37705
31426


In [53]:
print(f"Среднее отклонение линии рта для thinking: {sum(t_procents_mouth)/len(t_procents_mouth):.2f}%")

print(f"Среднее отклонение линии рта для feeling: {sum(f_procents_mouth)/len(f_procents_mouth):.2f}%")

Среднее отклонение линии рта для thinking: 17.69%
Среднее отклонение линии рта для feeling: 18.81%


In [54]:
print(f"Среднее отклонение бровей для thinking: {sum(t_procents_jawline)/len(t_procents_jawline):.2f}%")

print(f"Среднее отклонение бровей для feeling: {sum(f_procents_jawline)/len(f_procents_jawline):.2f}%")

Среднее отклонение бровей для thinking: 35.23%
Среднее отклонение бровей для feeling: 34.40%


In [ ]:
print(f"Среднее отклонение бровей для thinking: {sum(t_procents_brows)/len(t_procents_brows):.2f}%")

print(f"Среднее отклонение бровей для feeling: {sum(f_procents_brows)/len(f_procents_brows):.2f}%")